### A bigram is a contiguous sequence of two adjacent elements (items) in a given dataset or sequence. In natural language processing (NLP), particularly in the analysis of text, a bigram refers to a pair of consecutive words, characters, or tokens. Bigrams are used to capture local relationships or patterns within a sequence, and they are a type of n-gram where n is the number of elements in the sequence.

### For example, consider the sentence "The quick brown fox jumps." The bigrams in this sentence would be:

- The quick
- quick brown
- brown fox
- fox jumps
 jumps"

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])



In [5]:
#Example 
demo = 'Shoaib'
print('Original Text: ',demo)
print('Original Text Encoded: ',encode(demo))
print('Decoded back to Orignal Text: ',decode(encode(demo)))

Original Text:  Shoaib
Original Text Encoded:  [43, 61, 68, 54, 62, 55]
Decoded back to Orignal Text:  Shoaib


In [6]:
data = torch.tensor(encode(text), dtype=torch.long)

In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
def get_batch(split):
    block_size = 8
    batch_size = 4
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
x, y = get_batch('train')

In [10]:
x

tensor([[59,  1, 73, 61, 58,  1, 76, 68],
        [60, 71, 58, 54, 73,  1, 57, 58],
        [67, 60, 65, 58,  0, 69, 58, 71],
        [54, 67, 57,  1, 62, 67,  1, 54]], device='cuda:0')

In [11]:
y

tensor([[ 1, 73, 61, 58,  1, 76, 68, 68],
        [71, 58, 54, 73,  1, 57, 58, 54],
        [60, 65, 58,  0, 69, 58, 71, 72],
        [67, 57,  1, 62, 67,  1, 54,  1]], device='cuda:0')

In [12]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    

    def forward(self, index, targets=None):
        
        logits = self.token_embedding_table(index)
              
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
            
        return index

### Before Training

In [13]:
model  =BigramLanguageModel(vocab_size)
m = model.to(device)

In [14]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


?4;Y
Rjn!(7ySt-?A[dAZLx)eBOWZ
(F5l29hr0u6cpGEr&fWaRq[sASvBF4&ecEzkDKj]g9?]OTj6(kotCUN:""[rR:L3ACQrRPEK9
QNko!491Lnpub3S&ewj DCbvW5'yK6;"LQlgynp O6kHs2Zb9LoH:GEnIPE'xddMhrcinQn v1XSMOwj,?V)'N'b3QI7_CVHW7SuI2*yMLbV).TpfZcik"?JXpVYc&*f2C'3Pg,*
T?zxle[DvNMTE OWfD'2
(MNMLoM
RVyKl)v7a15eZH"MG"]YJraV"sS)'Z"HQrgEMa H,8'c7tDKhYqg8Dp*UxzAcfEgko2yG ;QfE645
0.)_TOKL9A&rRjYxhETS6B"P:0dm-4:uJ:q)vaR3PMb"U):le.8OAgwO3[zc4&xx0w?22v CVJQfqX1f;tDweB_hNVqiLK7rcYsw[6q[Sqg,fnqgVfTWs_VbwCJTD1(cETFnd-]]H!?q74gko,fJK[mw


In [15]:
@torch.inference_mode()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
learning_rate = 0.0001  #1e-2
eval_iters = 250
epochs = 20000
patience = 5  # Number of evaluations without improvement to tolerate

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
best_val_loss = float('inf')
current_patience = 0

for iter in range(epochs):
    
    if iter % eval_iters == 0:
        
        losses = estimate_loss()
        train_loss, val_loss = losses['train'], losses['val']
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            current_patience = 0
        else:
            current_patience += 1

        
        if current_patience >= patience:
            print(f'Early stopping after {iter} iterations due to improvement.')
            break
        
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # assigning to none it take less memory
    loss.backward()
    optimizer.step()
print('Final Loss: ',loss.item())


step: 0, train loss: 4.792, val loss: 4.732
step: 250, train loss: 4.744, val loss: 4.730
step: 500, train loss: 4.722, val loss: 4.697
step: 750, train loss: 4.683, val loss: 4.681
step: 1000, train loss: 4.651, val loss: 4.682
step: 1250, train loss: 4.663, val loss: 4.671
step: 1500, train loss: 4.630, val loss: 4.618
step: 1750, train loss: 4.625, val loss: 4.618
step: 2000, train loss: 4.588, val loss: 4.603
step: 2250, train loss: 4.585, val loss: 4.561
step: 2500, train loss: 4.566, val loss: 4.522
step: 2750, train loss: 4.520, val loss: 4.534
step: 3000, train loss: 4.534, val loss: 4.488
step: 3250, train loss: 4.509, val loss: 4.497
step: 3500, train loss: 4.475, val loss: 4.480
step: 3750, train loss: 4.466, val loss: 4.446
step: 4000, train loss: 4.411, val loss: 4.450
step: 4250, train loss: 4.433, val loss: 4.425
step: 4500, train loss: 4.408, val loss: 4.395
step: 4750, train loss: 4.403, val loss: 4.365
step: 5000, train loss: 4.371, val loss: 4.372
step: 5250, train l

### After Training

In [17]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


68Tol?zUSfWJNn9Zdn?_La??:alafujmN3dlw,fI6S!fo.V0fZ-514EE0[AYI O,4y2!4y"T-csR,jzk-_1s yijgaETrP56Wst3ckovOzQUf4G0EKj]

(Toft'zQ&sr,;b)pF:""TrPGH"Wsh-OxF_In.27,]qg)"wnZzS;qgm'toWiewvSUkJ7,fRa wuZH7re b_CFndaFxS)va**H7,QOG[jV7dypLx-ycIfj]G 0:&RKE-lAozhthqD4(TQA.3QZpYZ3&NY7uLp and_CQRAva al]R:ENYe DEKJ7qgr,9t4OvH8UPd-6'3NQ
Agevz[fx n!*68n KHarciX.zBK*.Bynd:io9F;IWE
rRQ0dk
G,.XZ;.b"TypJK!WT kok'3:a
mN0Vsa1L)g"F?.vBind
LoMT.w0ub_Dnddj?[TWAXLiE ShMg6tounbmoHHseta1Lp7MT4Rjv4dAB P)(nBC1yc!Qhin-caL3)6V3S

